In [9]:
import numpy as np
import multiset as ms

In [18]:
p =ms.Multiset([1,3,4,4,3,5,6]).difference([3])

In [38]:
def conformal_algorithm_from_old_examples(non_conformity_measure,previous_examples_list, example, error=.05):
    n = len(previous_examples_list)+1
    examples_list = previous_examples_list+[example]
    alpha = np.zeros(n)
    for i in range(n):
        bag = ms.Multiset(examples_list).difference([examples_list[i]])
        alpha[i] = non_conformity_measure(bag, examples_list[i])
    p_z = sum(alpha > alpha[n-1])/n
    return p_z >= error
        

In [20]:
list_fisher = [17,20,10,17,12,15,19,22,17,
               19,14,22,18,17,13,12,18,15,17]

In [46]:
def non_conformity_fisher(bag, example):
    n = len(bag)
    z_bar_bag = np.mean(bag)
    print(z_bar_bag*n)
    return(abs(((n*z_bar_bag+example)/(n+1))-example))

In [40]:
[conformal_algorithm_from_old_examples(non_conformity_fisher, list_fisher, i) for i in range(30)]

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False]

In [42]:
sum(list_fisher)

314